In [ ]:
import numpy as np
import time

import plotly.graph_objects as go 
from plotly.subplots import make_subplots

In [ ]:

def get_left_right_sides(n, p, lbd=1e-5):
    # matrices
    X = np.random.normal(
        loc=0, 
        scale=5**2,
        size=(n, p)
    )

    y = np.random.uniform(
        low=-1, 
        high=1, 
        size=(n, 1)
    )

    id_n = np.eye(n)
    id_p = np.eye(p)

    # operations and inv
    # left
    start_time = time.time()

    transpose_with_id_n = X @ X.T + lbd * id_n
    transpose_id_n_times_y = np.linalg.solve(transpose_with_id_n, y)
    left_side = X.T @ transpose_id_n_times_y

    left_duration = time.time() - start_time

    # right
    start_time = time.time()

    transpose_with_id_p = X.T @ X + lbd * id_p
    transpose_id_p_times_xy = np.linalg.solve(transpose_with_id_p, X.T @ y)  
    right_side = transpose_id_p_times_xy

    right_duration = time.time() - start_time

    return left_side, left_duration, right_side, right_duration

In [ ]:
# 
def get_duration(n, p):
    _, duration_left, _, duration_right = get_left_right_sides(n, p)

    return (duration_left, duration_right)


# samples of cases
case_a = [(i, i * 20) for i in range(100, 200+1, 10)]
case_b = [(i * 20, i) for i in range(100, 200+1, 10)]

  
times_case_a = list(
    map(
        lambda tup: get_duration(*tup), 
        case_a
    )
)

times_case_b = list(
    map(
        lambda tup: get_duration(*tup), 
        case_b
    )      
)

In [ ]:
# plot
fig = make_subplots(
    rows=1, 
    cols=2,
    subplot_titles=["duration for case n << p", "duration for case n >> p"]
)

color_name = {
    0: {
        "name": "left method",
        "color": '#636EFA'
    },
    1: {
        "name": "right method",
        "color": '#EF553B'
    }
}

fig.add_traces(data=[
    go.Bar(
        x=[str(el) for el in case_a],
        y=[tuple_duration[comp_method] for tuple_duration in times_case_a],
        marker_color=color_name[comp_method]["color"],
        name=color_name[comp_method]["name"],
    )
for comp_method in [0, 1]], 
rows=1, cols=1
)

fig.add_traces(data=[
    go.Bar(
        x=[str(el) for el in case_b],
        y=[tuple_duration[comp_method] for tuple_duration in times_case_b],
        marker_color=color_name[comp_method]["color"],
        name=color_name[comp_method]["name"],
        showlegend=False
    )
for comp_method in [0, 1]], 
rows=1, cols=2
)

# set layout
for i in [1, 2]:
    fig.update_yaxes(
        title_text="time", 
        type="log", 
        row=1, col=i
    )


fig.show()